In [1]:
import numpy as np
from torch import nn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
import sys
sys.path.append('/home/jamshedu/MLA-Final-Project/ANI-1_release/readers/lib/')
import pyanitools as pya

In [3]:
import pyanitools as pya

# Set the HDF5 file containing the data
hdf5file = 'ANI-1_release/ani_gdb_s01.h5'

# Construct the data loader class
data = pya.anidataloader(hdf5file)
data_iter = data.__iter__()
# Print the species of the data set one by one


In [4]:
mols = next(data_iter)
# Extract the data
P = mols['path']
X = mols['coordinates']
E = mols['energies']
S = mols['species']
sm = mols['smiles']

# Print the data
print("Path:   ", P)
print("  Smiles:      ","".join(sm))
print("  Symbols:     ", S)
print("  Coordinates: ", X.shape)
print("  Energies:    ", E.shape, "\n")

Path:    /gdb11_s01/gdb11_s01-0
  Smiles:       [H]C([H])([H])[H]
  Symbols:      ['C', 'H', 'H', 'H', 'H']
  Coordinates:  (5400, 5, 3)
  Energies:     (5400,) 



# Preliminary Information:

## Length of the coordinates is how many confirmations there are for each molecule with the final energies we need to predict.

## Shape of the coordinates is rows x columns x array. Where each array is a coordinate value x, y, z. The rows correspond to each confirmation and the columns correspond to each chemical species in the compound

## Working with the water molecule from s01 to show that the molecule can be read, train-test-split can be performed and batching occurs. Currently am calculating AEVs before conducting train-test-split, in the future may consider conducting train-test-split before calculating AEVs.

In [6]:
list_files_2 = ['ANI-1_release/ani_gdb_s01.h5']

data_set_X = []
y_energies = []

for i in list_files_2:
    hdf5file = i
    print(i)
    # Construct the data loader class
    data = pya.anidataloader(hdf5file)
    data_iter = data.__iter__()
    for mol in data_iter:  
        data_set_X.append({'species': mol['species'], 'coordinates': mol['coordinates']})
        y_energies.append(mol['energies'])

ANI-1_release/ani_gdb_s01.h5


In [7]:
data_set_X[0]

{'species': ['C', 'H', 'H', 'H', 'H'],
 'coordinates': array([[[-0.0034502 ,  0.01017081,  0.01938033],
         [-0.7954868 ,  0.5766599 , -0.5472012 ],
         [-0.39378393, -0.97992676,  0.2722862 ],
         [ 0.6344988 ,  0.4473651 ,  0.93568736],
         [ 0.59581804, -0.16517928, -0.8915708 ]],
 
        [[ 0.00311385, -0.02007288,  0.00282224],
         [-0.78331304,  0.7921426 , -0.26027855],
         [-0.45410746, -1.0295471 ,  0.31240797],
         [ 0.44713658,  0.63571125,  0.76770777],
         [ 0.7531731 , -0.1592813 , -0.85348135]],
 
        [[-0.0177954 , -0.02192003, -0.01129478],
         [-0.73020756,  0.6824338 , -0.30819252],
         [-0.382202  , -0.86773723,  0.3822439 ],
         [ 0.56933683,  0.50535196,  0.80417496],
         [ 0.75492465, -0.05902938, -0.74378157]],
 
        ...,
 
        [[ 0.00874467,  0.00214323, -0.00896647],
         [-0.85504687,  0.6273413 , -0.26081228],
         [-0.47105345, -0.9456285 ,  0.3252177 ],
         [ 0.48764503,

In [8]:
print(f'Length of final data set X is: {len(data_set_X)}')

Length of final data set X is: 3


## Symmetry Functions to calculate AEV for a particular atom in a molecule with a certain conformation

In [9]:
import numpy as np

def calc_f_C(Rij, RC):
    f_C_value = 0.5 * np.cos(np.pi * Rij / RC) + 0.5
    indicator = ((Rij <= RC) & (Rij != 0)).astype(float) # Make f_C(0)=0 to make sure the sum in distance conversion function and radial conversion function can run with j=i
    return f_C_value * indicator

def radial_component(Rijs, eta, Rs, RC=5.2):
    # Rijs is a 1d array, all other parameters are scalars
    f_C_values = calc_f_C(Rijs, RC)
    individual_components = np.exp(-eta * (Rijs - Rs) ** 2) * f_C_values
    return np.sum(individual_components)

def angular_component(Rij_vectors, Rik_vectors, zeta, theta_s, eta, Rs, RC=3.5):
    # Rij_vectors and Rik_vectors are 2d arrays with shape (n_atoms, 3), all other parameters are scalars
    # calculate theta_ijk values from vector operations
    dot_products = Rij_vectors.dot(Rik_vectors.T)
    Rij_norms = np.linalg.norm(Rij_vectors, axis=-1)
    Rik_norms = np.linalg.norm(Rik_vectors, axis=-1)
    norms = Rij_norms.reshape((-1, 1)).dot(Rik_norms.reshape((1, -1)))
    cos_values = np.clip(dot_products / (norms + 1e-8), -1, 1)
    theta_ijks = np.arccos(cos_values)
    theta_ijk_filter = (theta_ijks != 0).astype(float)
    mean_dists = (Rij_norms.reshape((-1, 1)) + Rik_norms.reshape((1, -1))) / 2
    f_C_values_Rij = calc_f_C(Rij_norms, RC)
    f_C_values_Rik = calc_f_C(Rik_norms, RC)
    f_C_values = f_C_values_Rij.reshape((-1, 1)).dot(f_C_values_Rik.reshape((1, -1)))
    individual_components = (1 + np.cos(theta_ijks - theta_s)) ** zeta * np.exp(-eta * (mean_dists - Rs) ** 2) * f_C_values * theta_ijk_filter
    return 2 ** (1 - zeta) * np.sum(individual_components)

def calc_aev(atom_types, coords, i_index):
    # atom_types are np.array of ints
    relative_coordinates = coords - coords[i_index]
    nearby_atom_indicator = np.linalg.norm(relative_coordinates, axis=-1) < 5.3
    relative_coordinates = relative_coordinates[nearby_atom_indicator]
    atom_types = atom_types[nearby_atom_indicator]
    radial_aev = np.array([radial_component(np.linalg.norm(relative_coordinates[atom_types == atom], axis=-1), eta, Rs) \
                           for atom in [0, 1, 2, 3] for eta in [16] \
                           for Rs in [0.900000,1.168750,1.437500,1.706250,1.975000,2.243750,2.51250,2.781250,3.050000,\
                                   3.318750,3.587500,3.856250,4.125000,4.39375,4.662500,4.931250]])
    angular_aev = np.array([angular_component(relative_coordinates[atom_types == atom_j], relative_coordinates[atom_types == atom_k],\
                                             zeta, theta_s, eta, Rs) \
                            for atom_j in [0, 1, 2, 3] for atom_k in range(atom_j, 4) for zeta in [32] \
                            for theta_s in [0.19634954,0.58904862,0.9817477,1.3744468,1.7671459,2.1598449,2.552544,2.945243]\
                            for eta in [8] for Rs in [0.900000,1.550000,2.200000,2.850000]])
    #print(len(radial_aev), len(angular_aev))
    return np.concatenate([radial_aev, angular_aev])

        

## Each AEV is 384 elements long and corresponds to 1 atom of a molecule in 1 conformation.
## So for s01, where there are 3 molecules, 
## CH4 (5400 conformations, 5 aevs/conformation), NH3 (3600 conformations, 4 aevs/conformation), H20 (1800, 3 aevs/conformation)
## total aevs = 27,000, 14,400, 5400
## For Water the shape we should use is (1800, 3, 384), 1800 conformations for (3 x 384)

In [15]:
water = data_set_X[2]
water_energies = y_energies[2]
water['coordinates'].shape

(1800, 3, 3)

## Function to convert coordinates to AEV for a molecule, with an output shape of (num_conformations, num_atoms, 384)

In [16]:
def convert_coordinates_to_aev(coordinates, species):
    mapping={"H":0, "C":1, "N":2, "O":3}
    elements = np.array([mapping[atom] for atom in species])
    print(elements)
    return_list = []
    for i in range(len(coordinates)):
        if i % 300 == 0:
            print(i)
        new_list = []
        for j in range(len(elements)):
            new_list.append(calc_aev(elements, coordinates[i], j))
        return_list.append(new_list)
    return np.array(return_list)

In [17]:
import time

t0 = time.time()
water_aevs = convert_coordinates_to_aev(water['coordinates'], water['species'])
t1 = time.time()
print(f'Total time taken for a water molecule is: {t1-t0}')

[3 0 0]
0
300
600
900
1200
1500
Total time taken for a water molecule is: 184.9922616481781


In [18]:
water_aevs.shape

(1800, 3, 384)

In [19]:
water_atoms = data_set_X[2]['species']
water_atoms

['O', 'H', 'H']

# Train Test Validation Splits

## Splitting the water_aevs (1800, 3, 384) and water_energies(1800,) into training, testing and validation sets below. These sets will be passed into our Trainer class to be further split into batches.

In [21]:
mol_x_train, mol_x_test, mol_y_train, mol_y_test = train_test_split(water_aevs, water_energies, test_size=1/3)

In [22]:
print(f'Input shape for molecule training: {mol_x_train.shape}')
print(f'Output shape for molecule training: {mol_y_train.shape}')
print(f'Input shape for molecule testing: {mol_x_test.shape}')
print(f'Output shape for molecule testing: {mol_y_test.shape}')

Input shape for molecule training: (1200, 3, 384)
Output shape for molecule training: (1200,)
Input shape for molecule testing: (600, 3, 384)
Output shape for molecule testing: (600,)


In [23]:
mol_x_train, mol_x_val, mol_y_train, mol_y_val = train_test_split(mol_x_train, mol_y_train, test_size=1/3)

In [24]:
print(f'Input shape for molecule training: {mol_x_train.shape}')
print(f'Output shape for molecule training: {mol_y_train.shape}')
print(f'Input shape for molecule validation: {mol_x_val.shape}')
print(f'Output shape for molecule validation: {mol_y_val.shape}')

Input shape for molecule training: (800, 3, 384)
Output shape for molecule training: (800,)
Input shape for molecule validation: (400, 3, 384)
Output shape for molecule validation: (400,)


In [114]:
# Look into splitting the molecule before into train test split and then calculate aevs and then send it into the training

## Creating a Preliminary Model

In [165]:
class ANI(nn.Module):
    def __init__(self):
        super(ANI, self).__init__()
        self.sub_nets = nn.ModuleDict({"C": ANI_sub(nn.Sequential(
                                                    nn.Linear(384,150),
                                                    nn.ReLU(),
                                                    nn.Linear(150, 50),
                                                    nn.ReLU(),
                                                    nn.Linear(50, 1)
                                                )), 
                                       "H": ANI_sub(nn.Sequential(
                                                    nn.Linear(384,150),
                                                    nn.ReLU(),
                                                    nn.Linear(150, 50),
                                                    nn.ReLU(),
                                                    nn.Linear(50, 1)
                                                )),                                         
                                       "N": ANI_sub(nn.Sequential(
                                                    nn.Linear(384,150),
                                                    nn.ReLU(),
                                                    nn.Linear(150, 50),
                                                    nn.ReLU(),
                                                    nn.Linear(50, 1)
                                                )), 
                                       "O": ANI_sub(nn.Sequential(
                                                    nn.Linear(384,150),
                                                    nn.ReLU(),
                                                    nn.Linear(150, 50),
                                                    nn.ReLU(),
                                                    nn.Linear(50, 1)
                                                ))})

    def forward(self, aevs, atom_types):
        atomic_energies = []
            
        for i in range(len(aevs)):
                
            intermediary_list = []
            
            for j in range(len(atom_types)):
                intermediary_list.append(self.sub_nets[atom_types[j]](aevs[i][j]))
                    # Makes a 1x3 vector
                
            atomic_energies.append(intermediary_list)
                # Makes an 1800 x 3 array
                
        # convert atomic energies to tensor and then sum up values in each row (sum all the columns for a row) 
        # to get an 1800 x 1 vector.
        atomic_energies = torch.tensor(atomic_energies, dtype=torch.float)
        
        # dim = 1 to sum on columns
        total_energies = torch.sum(atomic_energies,dim=1)
        print(total_energies)
        return total_energies

class ANI_sub(nn.Module):
    def __init__(self, architecture):
        super(ANI_sub, self).__init__()
        self.layers = architecture

    def forward(self, aev):
        aev = torch.tensor(aev, dtype=torch.float)
        atomic_energy = self.layers(aev)
        #print(atomic_energy)
        return atomic_energy


In [166]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r  took: %2.4f sec' % (f.__name__,  te-ts))
        return result
    return wrap

## Trainer Class that will convert Training and Validation sets of our data into batches for use in our model

In [167]:
from torch.optim import SGD, Adam
from torch import nn
import torch
import torch.nn.functional as F
import random
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

def create_chunks(complete_list, chunk_size=None, num_chunks=None):
    '''
    Cut a list into multiple chunks, each having chunk_size (the last chunk might be less than chunk_size) or having a total of num_chunk chunks
    '''
    chunks = []
    if num_chunks is None:
        num_chunks = math.ceil(len(complete_list) / chunk_size)
    elif chunk_size is None:
        chunk_size = math.ceil(len(complete_list) / num_chunks)
    for i in range(num_chunks):
        chunks.append(complete_list[i * chunk_size: (i + 1) * chunk_size])
    return chunks

class Trainer():
    def __init__(self, model, optimizer_type, learning_rate, epoch, batch_size):
        """ The class for training the model
        model: nn.Module
            A pytorch model
        optimizer_type: 'adam' or 'sgd'
        learning_rate: float
        epoch: int
        batch_size: int
        input_transform: func
            transforming input. Can do reshape here
        """
        self.model = model
        if optimizer_type == "sgd":
            self.optimizer = SGD(model.parameters(), learning_rate,momentum=0.9)
        elif optimizer_type == "adam":
            self.optimizer = Adam(model.parameters(), learning_rate)
            
        self.epoch = epoch
        self.batch_size = batch_size

    @timing
    def train(self, atom_types, inputs, outputs, val_inputs, val_outputs,early_stop=False,l2=False,silent=False):
        """ train self.model with specified arguments
        inputs: np.array, The shape of input_transform(input) should be (ndata,nfeatures) should be 32x32.
        outputs: np.array shape (ndata,)
        val_nputs: np.array, The shape of input_transform(val_input) should be (ndata,nfeatures)
        val_outputs: np.array shape (ndata,)
        early_stop: bool
        l2: bool
        silent: bool. Controls whether or not to print the train and val error during training
        
        @return
        a dictionary of arrays with train and val losses and accuracies
        """
        ### convert data to tensor of correct shape and type here ###
        #...
        #inputs = torch.tensor(inputs, dtype=torch.float)
        #outputs = torch.tensor(outputs, dtype=torch.int64)
                
        losses = []
        val_losses = []
        weights = self.model.state_dict()
        lowest_val_loss = np.inf
        
        loss_func = nn.MSELoss()
        
        for n_epoch in tqdm(range(self.epoch), leave=False):
            self.model.train()
            batch_indices = list(range(inputs.shape[0]))
            random.shuffle(batch_indices)
            batch_indices = create_chunks(batch_indices, chunk_size=self.batch_size)
            epoch_loss = 0
            #epoch_acc = 0
            for batch in batch_indices:
                batch_importance = len(batch) / len(outputs)
                batch_input = inputs[batch]
                batch_output = outputs[batch]
                ### make prediction and compute loss with loss function of your choice on this batch ###
                # self.model(aevs, atom_types)
                
                #print(batch_input.shape)
                batch_predictions = self.model(batch_input, atom_types)
                
                #print(batch_predictions.shape)

                #MSELoss here.
                loss = loss_func(batch_predictions, torch.tensor(batch_output, dtype=torch.float, requires_grad=True)) 
                #print(loss)
                if l2:
                    ### Compute the loss with L2 regularization ###
                    l2_norm = sum([p.pow(2.0).sum() for p in self.model.parameters()])
                    loss = loss + 0.00001 * l2_norm
                    #lambda is 1e-5 == 0.00001
                    
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                ### Compute epoch_loss 

                epoch_loss += loss.detach().item() * batch_importance

                
            val_loss = self.evaluate(atom_types, val_inputs, val_outputs)
            if n_epoch % 10 ==0 and not silent: 
                print("Epoch %d/%d - Loss: %.3f" % (n_epoch + 1, self.epoch, epoch_loss))
                print("              Val_loss: %.3f" % (val_loss))
                print("              Batch_size: %.3f" % (self.batch_size))
            losses.append(epoch_loss)
            val_losses.append(val_loss)
            if early_stop:
                if val_loss < lowest_val_loss:
                    lowest_val_loss = val_loss
                    weights = self.model.state_dict()

        if early_stop:
            self.model.load_state_dict(weights)    
        
        return {"losses": losses, "val_losses": val_losses}
        
    def evaluate(self, atom_types, inputs, outputs, print_loss=False):
        """ evaluate model on provided input and output
        inputs: np.array, The shape of input_transform(input) should be (ndata,nfeatures)
        outputs: np.array shape (ndata,)
        print_acc: bool
        
        @return
        losses: float
        acc: float
        """
        #inputs = torch.tensor(inputs, dtype=torch.float)
        #outputs = torch.tensor(outputs, dtype=torch.int64)
        
        loss_func = nn.MSELoss()
        
        self.model.eval()
        batch_indices = list(range(inputs.shape[0]))
        batch_indices = create_chunks(batch_indices, chunk_size=self.batch_size)
        losses = 0
        for batch in batch_indices:
            batch_importance = len(batch) / len(outputs)
            batch_input = inputs[batch]
            batch_output = outputs[batch]
            with torch.no_grad():
                ### Compute prediction and loss###
                batch_predictions = self.model(batch_input, atom_types)
                loss = loss_func(batch_predictions, torch.tensor(batch_output, dtype=torch.float, requires_grad=True))
                
                # apply l2 regularization
                '''
                l2_norm = sum([p.pow(2.0).sum() for p in self.model.parameters()])
                loss = loss + 0.00001 * l2_norm
                '''
                #lambda is 1e-5 == 0.00001
                
            losses += loss.detach().item() * batch_importance
        self.model.train()
        if print_loss:
            print("Loss: %.3f" % losses)
        return losses

In [168]:
ANI = ANI()

In [169]:
ANI_trainer = Trainer(ANI, "adam", 2e-3, 50, 128)

In [170]:
log=ANI_trainer.train(water_atoms, mol_x_train, mol_y_train, mol_x_val, mol_y_val)

  0%|                                                                                            | 0/50 [00:00<?, ?it/s]

tensor([-0.1606], grad_fn=<AddBackward0>)
tensor([0.0471], grad_fn=<AddBackward0>)
tensor([0.0472], grad_fn=<AddBackward0>)
tensor([-0.1538], grad_fn=<AddBackward0>)
tensor([0.0404], grad_fn=<AddBackward0>)
tensor([0.0414], grad_fn=<AddBackward0>)
tensor([-0.1644], grad_fn=<AddBackward0>)
tensor([0.0483], grad_fn=<AddBackward0>)
tensor([0.0475], grad_fn=<AddBackward0>)
tensor([-0.1595], grad_fn=<AddBackward0>)
tensor([0.0463], grad_fn=<AddBackward0>)
tensor([0.0453], grad_fn=<AddBackward0>)
tensor([-0.1622], grad_fn=<AddBackward0>)
tensor([0.0457], grad_fn=<AddBackward0>)
tensor([0.0441], grad_fn=<AddBackward0>)
tensor([-0.1638], grad_fn=<AddBackward0>)
tensor([0.0460], grad_fn=<AddBackward0>)
tensor([0.0456], grad_fn=<AddBackward0>)
tensor([-0.1631], grad_fn=<AddBackward0>)
tensor([0.0459], grad_fn=<AddBackward0>)
tensor([0.0479], grad_fn=<AddBackward0>)
tensor([-0.1582], grad_fn=<AddBackward0>)
tensor([0.0468], grad_fn=<AddBackward0>)
tensor([0.0477], grad_fn=<AddBackward0>)
tensor([

tensor([-0.1627], grad_fn=<AddBackward0>)
tensor([0.0460], grad_fn=<AddBackward0>)
tensor([0.0477], grad_fn=<AddBackward0>)
tensor([-0.1653], grad_fn=<AddBackward0>)
tensor([0.0456], grad_fn=<AddBackward0>)
tensor([0.0469], grad_fn=<AddBackward0>)
tensor([-0.1638], grad_fn=<AddBackward0>)
tensor([0.0479], grad_fn=<AddBackward0>)
tensor([0.0473], grad_fn=<AddBackward0>)
tensor([-0.1571], grad_fn=<AddBackward0>)
tensor([0.0429], grad_fn=<AddBackward0>)
tensor([0.0434], grad_fn=<AddBackward0>)
tensor([-0.1640], grad_fn=<AddBackward0>)
tensor([0.0477], grad_fn=<AddBackward0>)
tensor([0.0472], grad_fn=<AddBackward0>)
tensor([-0.1584], grad_fn=<AddBackward0>)
tensor([0.0456], grad_fn=<AddBackward0>)
tensor([0.0459], grad_fn=<AddBackward0>)
tensor([-0.1656], grad_fn=<AddBackward0>)
tensor([0.0474], grad_fn=<AddBackward0>)
tensor([0.0496], grad_fn=<AddBackward0>)
tensor([-0.1590], grad_fn=<AddBackward0>)
tensor([0.0459], grad_fn=<AddBackward0>)
tensor([0.0436], grad_fn=<AddBackward0>)
tensor([

tensor([0.0451], grad_fn=<AddBackward0>)
tensor([-0.1656], grad_fn=<AddBackward0>)
tensor([0.0465], grad_fn=<AddBackward0>)
tensor([0.0473], grad_fn=<AddBackward0>)
tensor([-0.1641], grad_fn=<AddBackward0>)
tensor([0.0470], grad_fn=<AddBackward0>)
tensor([0.0479], grad_fn=<AddBackward0>)
tensor([-0.1601], grad_fn=<AddBackward0>)
tensor([0.0459], grad_fn=<AddBackward0>)
tensor([0.0447], grad_fn=<AddBackward0>)
tensor([-0.1608], grad_fn=<AddBackward0>)
tensor([0.0472], grad_fn=<AddBackward0>)
tensor([0.0483], grad_fn=<AddBackward0>)
tensor([-0.1635], grad_fn=<AddBackward0>)
tensor([0.0455], grad_fn=<AddBackward0>)
tensor([0.0461], grad_fn=<AddBackward0>)
tensor([-0.1647], grad_fn=<AddBackward0>)
tensor([0.0492], grad_fn=<AddBackward0>)
tensor([0.0479], grad_fn=<AddBackward0>)
tensor([-0.1626], grad_fn=<AddBackward0>)
tensor([0.0459], grad_fn=<AddBackward0>)
tensor([0.0477], grad_fn=<AddBackward0>)
tensor([-0.1645], grad_fn=<AddBackward0>)
tensor([0.0471], grad_fn=<AddBackward0>)
tensor([

tensor([0.0445], grad_fn=<AddBackward0>)
tensor([0.0438], grad_fn=<AddBackward0>)
tensor([-0.1624], grad_fn=<AddBackward0>)
tensor([0.0482], grad_fn=<AddBackward0>)
tensor([0.0455], grad_fn=<AddBackward0>)
tensor([-0.1643], grad_fn=<AddBackward0>)
tensor([0.0473], grad_fn=<AddBackward0>)
tensor([0.0456], grad_fn=<AddBackward0>)
tensor([-0.1599], grad_fn=<AddBackward0>)
tensor([0.0452], grad_fn=<AddBackward0>)
tensor([0.0456], grad_fn=<AddBackward0>)
tensor([-0.1640], grad_fn=<AddBackward0>)
tensor([0.0469], grad_fn=<AddBackward0>)
tensor([0.0442], grad_fn=<AddBackward0>)
tensor([-0.1593], grad_fn=<AddBackward0>)
tensor([0.0422], grad_fn=<AddBackward0>)
tensor([0.0444], grad_fn=<AddBackward0>)
tensor([-0.1541], grad_fn=<AddBackward0>)
tensor([0.0445], grad_fn=<AddBackward0>)
tensor([0.0436], grad_fn=<AddBackward0>)
tensor([-0.1564], grad_fn=<AddBackward0>)
tensor([0.0468], grad_fn=<AddBackward0>)
tensor([0.0460], grad_fn=<AddBackward0>)
tensor([-0.1657], grad_fn=<AddBackward0>)
tensor([

...

tensor([-0.1599], grad_fn=<AddBackward0>)
tensor([0.0452], grad_fn=<AddBackward0>)
tensor([0.0456], grad_fn=<AddBackward0>)
tensor([-0.1635], grad_fn=<AddBackward0>)
tensor([0.0484], grad_fn=<AddBackward0>)
tensor([0.0477], grad_fn=<AddBackward0>)
tensor([-0.1546], grad_fn=<AddBackward0>)
tensor([0.0432], grad_fn=<AddBackward0>)
tensor([0.0441], grad_fn=<AddBackward0>)
tensor([-0.1635], grad_fn=<AddBackward0>)
tensor([0.0477], grad_fn=<AddBackward0>)
tensor([0.0463], grad_fn=<AddBackward0>)
tensor([-0.1540], grad_fn=<AddBackward0>)
tensor([0.0416], grad_fn=<AddBackward0>)
tensor([0.0425], grad_fn=<AddBackward0>)
tensor([-0.1577], grad_fn=<AddBackward0>)
tensor([0.0455], grad_fn=<AddBackward0>)
tensor([0.0449], grad_fn=<AddBackward0>)
tensor([-0.1659], grad_fn=<AddBackward0>)
tensor([0.0489], grad_fn=<AddBackward0>)
tensor([0.0471], grad_fn=<AddBackward0>)
tensor([-0.1641], grad_fn=<AddBackward0>)
tensor([0.0470], grad_fn=<AddBackward0>)
tensor([0.0479], grad_fn=<AddBackward0>)
tensor([

tensor([0.0459], grad_fn=<AddBackward0>)
tensor([-0.1638], grad_fn=<AddBackward0>)
tensor([0.0462], grad_fn=<AddBackward0>)
tensor([0.0453], grad_fn=<AddBackward0>)
tensor([-0.1646], grad_fn=<AddBackward0>)
tensor([0.0480], grad_fn=<AddBackward0>)
tensor([0.0469], grad_fn=<AddBackward0>)
tensor([-0.1588], grad_fn=<AddBackward0>)
tensor([0.0473], grad_fn=<AddBackward0>)
tensor([0.0475], grad_fn=<AddBackward0>)
tensor([-0.1562], grad_fn=<AddBackward0>)
tensor([0.0465], grad_fn=<AddBackward0>)
tensor([0.0458], grad_fn=<AddBackward0>)
tensor([-0.1651], grad_fn=<AddBackward0>)
tensor([0.0470], grad_fn=<AddBackward0>)
tensor([0.0464], grad_fn=<AddBackward0>)
tensor([-0.1549], grad_fn=<AddBackward0>)
tensor([0.0453], grad_fn=<AddBackward0>)
tensor([0.0445], grad_fn=<AddBackward0>)
tensor([-0.1641], grad_fn=<AddBackward0>)
tensor([0.0454], grad_fn=<AddBackward0>)
tensor([0.0463], grad_fn=<AddBackward0>)
tensor([-0.1642], grad_fn=<AddBackward0>)
tensor([0.0457], grad_fn=<AddBackward0>)
tensor([

tensor([0.0465], grad_fn=<AddBackward0>)
tensor([-0.1653], grad_fn=<AddBackward0>)
tensor([0.0476], grad_fn=<AddBackward0>)
tensor([0.0495], grad_fn=<AddBackward0>)
tensor([-0.1645], grad_fn=<AddBackward0>)
tensor([0.0482], grad_fn=<AddBackward0>)
tensor([0.0452], grad_fn=<AddBackward0>)
tensor([-0.1633], grad_fn=<AddBackward0>)
tensor([0.0475], grad_fn=<AddBackward0>)
tensor([0.0478], grad_fn=<AddBackward0>)
tensor([-0.1620], grad_fn=<AddBackward0>)
tensor([0.0469], grad_fn=<AddBackward0>)
tensor([0.0474], grad_fn=<AddBackward0>)
tensor([-0.1654], grad_fn=<AddBackward0>)
tensor([0.0456], grad_fn=<AddBackward0>)
tensor([0.0481], grad_fn=<AddBackward0>)
tensor([-0.1626], grad_fn=<AddBackward0>)
tensor([0.0441], grad_fn=<AddBackward0>)
tensor([0.0427], grad_fn=<AddBackward0>)
tensor([-0.1637], grad_fn=<AddBackward0>)
tensor([0.0462], grad_fn=<AddBackward0>)
tensor([0.0483], grad_fn=<AddBackward0>)
tensor([-0.1627], grad_fn=<AddBackward0>)
tensor([0.0474], grad_fn=<AddBackward0>)
tensor([

tensor([0.0451], grad_fn=<AddBackward0>)
tensor([-0.1558], grad_fn=<AddBackward0>)
tensor([0.0458], grad_fn=<AddBackward0>)
tensor([0.0442], grad_fn=<AddBackward0>)
tensor([-0.1573], grad_fn=<AddBackward0>)
tensor([0.0470], grad_fn=<AddBackward0>)
tensor([0.0447], grad_fn=<AddBackward0>)
tensor([-0.1642], grad_fn=<AddBackward0>)
tensor([0.0457], grad_fn=<AddBackward0>)
tensor([0.0465], grad_fn=<AddBackward0>)
tensor([-0.1643], grad_fn=<AddBackward0>)
tensor([0.0492], grad_fn=<AddBackward0>)
tensor([0.0461], grad_fn=<AddBackward0>)
tensor([-0.1612], grad_fn=<AddBackward0>)
tensor([0.0429], grad_fn=<AddBackward0>)
tensor([0.0432], grad_fn=<AddBackward0>)
tensor([-0.1645], grad_fn=<AddBackward0>)
tensor([0.0484], grad_fn=<AddBackward0>)
tensor([0.0476], grad_fn=<AddBackward0>)
tensor([-0.1619], grad_fn=<AddBackward0>)
tensor([0.0478], grad_fn=<AddBackward0>)
tensor([0.0471], grad_fn=<AddBackward0>)
tensor([-0.1639], grad_fn=<AddBackward0>)
tensor([0.0493], grad_fn=<AddBackward0>)
tensor([

KeyboardInterrupt: 

In [158]:
log

{'losses': [5813.173984375,
  5813.173984375,
  5813.1740234375,
  5813.17421875,
  5813.17412109375,
  5813.174277343751,
  5813.17404296875,
  5813.174296875,
  5813.174277343749,
  5813.1738867187505,
  5813.1739453125,
  5813.174062499999,
  5813.174082031251,
  5813.173828125,
  5813.174140625,
  5813.17408203125,
  5813.174042968751,
  5813.173984375001,
  5813.1741015625,
  5813.173984375,
  5813.17416015625,
  5813.1744140625,
  5813.17416015625,
  5813.1742578125,
  5813.17427734375,
  5813.174140625,
  5813.174140625,
  5813.1740234375,
  5813.174316406249,
  5813.1742968749995,
  5813.1740234375,
  5813.174140624999,
  5813.174121093751,
  5813.174199218751,
  5813.174042968751,
  5813.173984375001,
  5813.174140625,
  5813.173925781249,
  5813.173867187499,
  5813.1740625,
  5813.1741015625,
  5813.17388671875,
  5813.17416015625,
  5813.17421875,
  5813.1743359375005,
  5813.174062500001,
  5813.17421875,
  5813.17404296875,
  5813.173828125,
  5813.17404296875],
 'val_los

In [132]:
x, y = water_aevs[:][2].shape
print(x)
print(y)

3
384


In [127]:
reshaped_water = water_aevs.reshape(3, 1800, 384)

In [131]:
reshaped_water[1].shape

(1800, 384)